# C-code to Python code


In [1]:
class A5_1:
    R1MASK = 0x07FFFF  # 19 bits, numbered 0..18
    R2MASK = 0x3FFFFF  # 22 bits, numbered 0..21
    R3MASK = 0x7FFFFF  # 23 bits, numbered 0..22

    R1MID = 0x000100  # bit 8
    R2MID = 0x000400  # bit 10
    R3MID = 0x000400  # bit 10

    R1TAPS = 0x072000  # bits 18,17,16,13
    R2TAPS = 0x300000  # bits 21,20
    R3TAPS = 0x700080  # bits 22,21,20,7

    R1OUT = 0x040000  # bit 18 (the high bit)
    R2OUT = 0x200000  # bit 21 (the high bit)
    R3OUT = 0x400000  # bit 22 (the high bit)

    def __init__(self):
        self.R1 = 0
        self.R2 = 0
        self.R3 = 0

    def parity(self, x):
        x ^= x >> 16
        x ^= x >> 8
        x ^= x >> 4
        x ^= x >> 2
        x ^= x >> 1
        return x & 1

    def clockone(self, reg, mask, taps):
        t = reg & taps
        reg = (reg << 1) & mask
        reg |= self.parity(t)
        return reg

    def majority(self):
        sum = self.parity(self.R1 & self.R1MID) + self.parity(self.R2 & self.R2MID) + self.parity(self.R3 & self.R3MID)
        return 1 if sum >= 2 else 0

    def clock(self):
        maj = self.majority()
        if ((self.R1 & self.R1MID) != 0) == maj:
            self.R1 = self.clockone(self.R1, self.R1MASK, self.R1TAPS)
        if ((self.R2 & self.R2MID) != 0) == maj:
            self.R2 = self.clockone(self.R2, self.R2MASK, self.R2TAPS)
        if ((self.R3 & self.R3MID) != 0) == maj:
            self.R3 = self.clockone(self.R3, self.R3MASK, self.R3TAPS)

    def clockallthree(self):
        self.R1 = self.clockone(self.R1, self.R1MASK, self.R1TAPS)
        self.R2 = self.clockone(self.R2, self.R2MASK, self.R2TAPS)
        self.R3 = self.clockone(self.R3, self.R3MASK, self.R3TAPS)

    def getbit(self):
        return self.parity(self.R1 & self.R1OUT) ^ self.parity(self.R2 & self.R2OUT) ^ self.parity(self.R3 & self.R3OUT)

    def keysetup(self, key, frame):
        self.R1 = self.R2 = self.R3 = 0

        for i in range(64):
            self.clockallthree()
            keybit = (key[i // 8] >> (i & 7)) & 1
            self.R1 ^= keybit
            self.R2 ^= keybit
            self.R3 ^= keybit

        for i in range(22):
            self.clockallthree()
            framebit = (frame >> i) & 1
            self.R1 ^= framebit
            self.R2 ^= framebit
            self.R3 ^= framebit

        for _ in range(100):
            self.clock()

    def run(self):
        AtoBkeystream = [0] * 15
        BtoAkeystream = [0] * 15

        for i in range(114):
            self.clock()
            AtoBkeystream[i // 8] |= self.getbit() << (7 - (i & 7))

        for i in range(114):
            self.clock()
            BtoAkeystream[i // 8] |= self.getbit() << (7 - (i & 7))

        return AtoBkeystream, BtoAkeystream

def test():
    key = [0x12, 0x23, 0x45, 0x67, 0x89, 0xAB, 0xCD, 0xEF]
    frame = 0x134
    goodAtoB = [0x53, 0x4E, 0xAA, 0x58, 0x2F, 0xE8, 0x15, 0x1A, 0xB6, 0xE1, 0x85, 0x5A, 0x72, 0x8C, 0x00]
    goodBtoA = [0x24, 0xFD, 0x35, 0xA3, 0x5D, 0x5F, 0xB6, 0x52, 0x6D, 0x32, 0xF9, 0x06, 0xDF, 0x1A, 0xC0]

    a5_1 = A5_1()
    a5_1.keysetup(key, frame)
    AtoB, BtoA = a5_1.run()

    print("key: 0x", ''.join(f"{b:02X}" for b in key))
    print("frame number: 0x%06X" % frame)
    print("known good output:")
    print(" A->B: 0x", ''.join(f"{b:02X}" for b in goodAtoB))
    print("  B->A: 0x", ''.join(f"{b:02X}" for b in goodBtoA))
    print("observed output:")
    print(" A->B: 0x", ''.join(f"{b:02X}" for b in AtoB))
    print("  B->A: 0x", ''.join(f"{b:02X}" for b in BtoA))

    if AtoB == goodAtoB and BtoA == goodBtoA:
        print("Self-check succeeded: everything looks ok.")
    else:
        print("Self-check failed: output does not match the known-good test vector.")

if __name__ == "__main__":
    test()


key: 0x 1223456789ABCDEF
frame number: 0x000134
known good output:
 A->B: 0x 534EAA582FE8151AB6E1855A728C00
  B->A: 0x 24FD35A35D5FB6526D32F906DF1AC0
observed output:
 A->B: 0x 534EAA582FE8151AB6E1855A728C00
  B->A: 0x 24FD35A35D5FB6526D32F906DF1AC0
Self-check succeeded: everything looks ok.


# Modified Code for generating 10 byte Keystream

In [4]:
class A5_1:
    R1MASK = 0x07FFFF  # 19 bits, numbered 0..18
    R2MASK = 0x3FFFFF  # 22 bits, numbered 0..21
    R3MASK = 0x7FFFFF  # 23 bits, numbered 0..22

    R1MID = 0x000100  # bit 8
    R2MID = 0x000400  # bit 10
    R3MID = 0x000400  # bit 10

    R1TAPS = 0x072000  # bits 18,17,16,13
    R2TAPS = 0x300000  # bits 21,20
    R3TAPS = 0x700080  # bits 22,21,20,7

    R1OUT = 0x040000  # bit 18 (the high bit)
    R2OUT = 0x200000  # bit 21 (the high bit)
    R3OUT = 0x400000  # bit 22 (the high bit)

    def __init__(self):
        self.R1 = 0
        self.R2 = 0
        self.R3 = 0

    def parity(self, x):
        x ^= x >> 16
        x ^= x >> 8
        x ^= x >> 4
        x ^= x >> 2
        x ^= x >> 1
        return x & 1

    def clockone(self, reg, mask, taps):
        t = reg & taps
        reg = (reg << 1) & mask
        reg |= self.parity(t)
        return reg

    def majority(self):
        sum = self.parity(self.R1 & self.R1MID) + self.parity(self.R2 & self.R2MID) + self.parity(self.R3 & self.R3MID)
        return 1 if sum >= 2 else 0

    def clock(self):
        maj = self.majority()
        if ((self.R1 & self.R1MID) != 0) == maj:
            self.R1 = self.clockone(self.R1, self.R1MASK, self.R1TAPS)
        if ((self.R2 & self.R2MID) != 0) == maj:
            self.R2 = self.clockone(self.R2, self.R2MASK, self.R2TAPS)
        if ((self.R3 & self.R3MID) != 0) == maj:
            self.R3 = self.clockone(self.R3, self.R3MASK, self.R3TAPS)

    def clockallthree(self):
        self.R1 = self.clockone(self.R1, self.R1MASK, self.R1TAPS)
        self.R2 = self.clockone(self.R2, self.R2MASK, self.R2TAPS)
        self.R3 = self.clockone(self.R3, self.R3MASK, self.R3TAPS)

    def getbit(self):
        return self.parity(self.R1 & self.R1OUT) ^ self.parity(self.R2 & self.R2OUT) ^ self.parity(self.R3 & self.R3OUT)

    def keysetup(self, key, frame):
        self.R1 = self.R2 = self.R3 = 0

        for i in range(64):
            self.clockallthree()
            keybit = (key[i // 8] >> (i & 7)) & 1
            self.R1 ^= keybit
            self.R2 ^= keybit
            self.R3 ^= keybit

        for i in range(22):
            self.clockallthree()
            framebit = (frame >> i) & 1
            self.R1 ^= framebit
            self.R2 ^= framebit
            self.R3 ^= framebit

        for _ in range(100):
            self.clock()

    def generate_keystream(self, length):
        keystream = [0] * length

        for i in range(length * 8):
            self.clock()
            keystream[i // 8] |= self.getbit() << (7 - (i & 7))

        return keystream

def test():
    key = [0x12, 0x23, 0x45, 0x67, 0x89, 0xAB, 0xCD, 0xEF]
    frame = 0x134
    good_keystream = [0x53, 0x4E, 0xAA, 0x58, 0x2F, 0xE8, 0x15, 0x1A, 0xB6, 0xE1]  # First 10 bytes of A->B keystream

    a5_1 = A5_1()
    a5_1.keysetup(key, frame)
    keystream = a5_1.generate_keystream(10)

    print("key: 0x", ''.join(f"{b:02X}" for b in key))
    print("frame number: 0x%06X" % frame)
    print("known good output:")
    print(" keystream: 0x", ''.join(f"{b:02X}" for b in good_keystream))
    print("observed output:")
    print(" keystream: 0x", ''.join(f"{b:02X}" for b in keystream))

    if keystream == good_keystream:
        print("Self-check succeeded: everything looks ok.")
    else:
        print("Self-check failed: output does not match the known-good test vector.")

if __name__ == "__main__":
    test()


key: 0x 1223456789ABCDEF
frame number: 0x000134
known good output:
 keystream: 0x 534EAA582FE8151AB6E1
observed output:
 keystream: 0x 534EAA582FE8151AB6E1
Self-check succeeded: everything looks ok.


# Code for generating CSV file

In [1]:
import csv
import random

class A5_1:
    R1MASK = 0x07FFFF  # 19 bits, numbered 0..18
    R2MASK = 0x3FFFFF  # 22 bits, numbered 0..21
    R3MASK = 0x7FFFFF  # 23 bits, numbered 0..22

    R1MID = 0x000100  # bit 8
    R2MID = 0x000400  # bit 10
    R3MID = 0x000400  # bit 10

    R1TAPS = 0x072000  # bits 18,17,16,13
    R2TAPS = 0x300000  # bits 21,20
    R3TAPS = 0x700080  # bits 22,21,20,7

    R1OUT = 0x040000  # bit 18 (the high bit)
    R2OUT = 0x200000  # bit 21 (the high bit)
    R3OUT = 0x400000  # bit 22 (the high bit)

    def __init__(self):
        self.R1 = 0
        self.R2 = 0
        self.R3 = 0

    def parity(self, x):
        x ^= x >> 16
        x ^= x >> 8
        x ^= x >> 4
        x ^= x >> 2
        x ^= x >> 1
        return x & 1

    def clockone(self, reg, mask, taps):
        t = reg & taps
        reg = (reg << 1) & mask
        reg |= self.parity(t)
        return reg

    def majority(self):
        sum = self.parity(self.R1 & self.R1MID) + self.parity(self.R2 & self.R2MID) + self.parity(self.R3 & self.R3MID)
        return 1 if sum >= 2 else 0

    def clock(self):
        maj = self.majority()
        if ((self.R1 & self.R1MID) != 0) == maj:
            self.R1 = self.clockone(self.R1, self.R1MASK, self.R1TAPS)
        if ((self.R2 & self.R2MID) != 0) == maj:
            self.R2 = self.clockone(self.R2, self.R2MASK, self.R2TAPS)
        if ((self.R3 & self.R3MID) != 0) == maj:
            self.R3 = self.clockone(self.R3, self.R3MASK, self.R3TAPS)

    def clockallthree(self):
        self.R1 = self.clockone(self.R1, self.R1MASK, self.R1TAPS)
        self.R2 = self.clockone(self.R2, self.R2MASK, self.R2TAPS)
        self.R3 = self.clockone(self.R3, self.R3MASK, self.R3TAPS)

    def getbit(self):
        return self.parity(self.R1 & self.R1OUT) ^ self.parity(self.R2 & self.R2OUT) ^ self.parity(self.R3 & self.R3OUT)

    def keysetup(self, key, frame):
        self.R1 = self.R2 = self.R3 = 0

        for i in range(64):
            self.clockallthree()
            keybit = (key[i // 8] >> (i & 7)) & 1
            self.R1 ^= keybit
            self.R2 ^= keybit
            self.R3 ^= keybit

        for i in range(22):
            self.clockallthree()
            framebit = (frame >> i) & 1
            self.R1 ^= framebit
            self.R2 ^= framebit
            self.R3 ^= framebit

        for _ in range(100):
            self.clock()

    def generate_keystream(self, length):
        keystream = [0] * length

        for i in range(length * 8):
            self.clock()
            keystream[i // 8] |= self.getbit() << (7 - (i & 7))

        return keystream
def generate_random_key():
    return [random.randint(0, 255) for _ in range(8)]
    
def convert_key_to_bits(key):
    key_bits = []
    for byte in key:
        for bit in range(8):
            key_bits.append((byte >> bit) & 1)
    return key_bits

def generate_records(filename, num_records):
    frame = 0x134  # Using a fixed frame number for simplicity

    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        headers = [f'key_bit_{i}' for i in range(64)] + [f'keystream_byte_{i}' for i in range(10)]
        csvwriter.writerow(headers)

        for _ in range(num_records):
            key = generate_random_key()
            key_bits = convert_key_to_bits(key)

            # Setup the cipher
            a5_1 = A5_1()
            a5_1.keysetup(key, frame)
            
            # Generate 10 bytes of keystream
            keystream = a5_1.generate_keystream(10)
            
            
            # Create the record and write it to the CSV
            record = key_bits + keystream
            csvwriter.writerow(record)

if __name__ == "__main__":
    generate_records('keystream_records_10lakh.csv', 1500000)


# Initial key is BYTE instead of bits

In [2]:
import csv
import random

class A5_1:
    R1MASK = 0x07FFFF  # 19 bits, numbered 0..18
    R2MASK = 0x3FFFFF  # 22 bits, numbered 0..21
    R3MASK = 0x7FFFFF  # 23 bits, numbered 0..22

    R1MID = 0x000100  # bit 8
    R2MID = 0x000400  # bit 10
    R3MID = 0x000400  # bit 10

    R1TAPS = 0x072000  # bits 18,17,16,13
    R2TAPS = 0x300000  # bits 21,20
    R3TAPS = 0x700080  # bits 22,21,20,7

    R1OUT = 0x040000  # bit 18 (the high bit)
    R2OUT = 0x200000  # bit 21 (the high bit)
    R3OUT = 0x400000  # bit 22 (the high bit)

    def __init__(self):
        self.R1 = 0
        self.R2 = 0
        self.R3 = 0

    def parity(self, x):
        x ^= x >> 16
        x ^= x >> 8
        x ^= x >> 4
        x ^= x >> 2
        x ^= x >> 1
        return x & 1

    def clockone(self, reg, mask, taps):
        t = reg & taps
        reg = (reg << 1) & mask
        reg |= self.parity(t)
        return reg

    def majority(self):
        sum = self.parity(self.R1 & self.R1MID) + self.parity(self.R2 & self.R2MID) + self.parity(self.R3 & self.R3MID)
        return 1 if sum >= 2 else 0

    def clock(self):
        maj = self.majority()
        if ((self.R1 & self.R1MID) != 0) == maj:
            self.R1 = self.clockone(self.R1, self.R1MASK, self.R1TAPS)
        if ((self.R2 & self.R2MID) != 0) == maj:
            self.R2 = self.clockone(self.R2, self.R2MASK, self.R2TAPS)
        if ((self.R3 & self.R3MID) != 0) == maj:
            self.R3 = self.clockone(self.R3, self.R3MASK, self.R3TAPS)

    def clockallthree(self):
        self.R1 = self.clockone(self.R1, self.R1MASK, self.R1TAPS)
        self.R2 = self.clockone(self.R2, self.R2MASK, self.R2TAPS)
        self.R3 = self.clockone(self.R3, self.R3MASK, self.R3TAPS)

    def getbit(self):
        return self.parity(self.R1 & self.R1OUT) ^ self.parity(self.R2 & self.R2OUT) ^ self.parity(self.R3 & self.R3OUT)

    def keysetup(self, key, frame):
        self.R1 = self.R2 = self.R3 = 0

        for i in range(64):
            self.clockallthree()
            keybit = (key[i // 8] >> (i & 7)) & 1
            self.R1 ^= keybit
            self.R2 ^= keybit
            self.R3 ^= keybit

        for i in range(22):
            self.clockallthree()
            framebit = (frame >> i) & 1
            self.R1 ^= framebit
            self.R2 ^= framebit
            self.R3 ^= framebit

        for _ in range(100):
            self.clock()

    def generate_keystream(self, length):
        keystream = [0] * length

        for i in range(length * 8):
            self.clock()
            keystream[i // 8] |= self.getbit() << (7 - (i & 7))

        return keystream
def generate_random_key():
    return [random.randint(0, 255) for _ in range(8)]
    
def convert_key_to_bits(key):
    key_bits = []
    for byte in key:
        for bit in range(8):
            key_bits.append((byte >> bit) & 1)
    return key_bits

def generate_records(filename, num_records):
    frame = 0x134  # Using a fixed frame number for simplicity

    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        headers = [f'key_byte_{i}' for i in range(8)] + [f'keystream_byte_{i}' for i in range(10)]
        csvwriter.writerow(headers)

        for _ in range(num_records):
            key = generate_random_key()

            # Setup the cipher
            a5_1 = A5_1()
            a5_1.keysetup(key, frame)
            
            # Generate 10 bytes of keystream
            keystream = a5_1.generate_keystream(10)
            
            # Create the record and write it to the CSV
            record = key + keystream
            csvwriter.writerow(record)


if __name__ == "__main__":
    generate_records('keystream_records_4lakh.csv',400000)


# Testing a model on 0th byte of keystream

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np


In [7]:
# Load the data from CSV
data = pd.read_csv('keystream_records.csv')

# Separate the input features (64-bit key) and the target (0th byte of keystream)
X = data.iloc[:, :64].values  # First 64 columns are the 64-bit key
y = data.iloc[:, 64].values   # 65th column is the 0th byte of the keystream

# Convert the target to categorical (byte values range from 0 to 255)
y = y.astype(np.uint8)


In [10]:
# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
class KeystreamClassifier(nn.Module):
    def __init__(self):
        super(KeystreamClassifier, self).__init__()
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 256)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
model = KeystreamClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create data loaders
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)


In [13]:
# Training the model
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')


Epoch 1/20, Loss: 5.546680651473999
Epoch 2/20, Loss: 5.5437264659881595
Epoch 3/20, Loss: 5.536945764160156
Epoch 4/20, Loss: 5.517007850265503
Epoch 5/20, Loss: 5.479582009506226
Epoch 6/20, Loss: 5.4277951171875
Epoch 7/20, Loss: 5.365756849288941
Epoch 8/20, Loss: 5.298566541671753
Epoch 9/20, Loss: 5.23156375579834
Epoch 10/20, Loss: 5.167142416763306
Epoch 11/20, Loss: 5.10946356048584
Epoch 12/20, Loss: 5.05725241394043
Epoch 13/20, Loss: 5.010503924942016
Epoch 14/20, Loss: 4.9690794734954835
Epoch 15/20, Loss: 4.931884042358399
Epoch 16/20, Loss: 4.8995677173614505
Epoch 17/20, Loss: 4.870518451690674
Epoch 18/20, Loss: 4.842836697006225
Epoch 19/20, Loss: 4.819706113052368
Epoch 20/20, Loss: 4.797079890060425


In [14]:
# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test, predicted.numpy())
    print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 0.43%


In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


print(tf.__version__)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Load the data from CSV
data = pd.read_csv('keystream_records_10lakh.csv')

# Separate the input features (64-bit key) and the targets (10 bytes of keystream)
X = data.iloc[:, :64].values  # First 64 columns are the 64-bit key
y = data.iloc[:, 64:74].values  # Next 10 columns are the 10 bytes of the keystream

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape X to be (num_samples, 64, 1) for Conv1D
X_train_reshaped = X_train.reshape(-1, 64, 1)
X_test_reshaped = X_test.reshape(-1, 64, 1)

# Function to build the model
def build_model():
    input_layer = Input(shape=(64, 1))
    x = Conv1D(32, 4, activation='relu', padding='same', strides=2)(input_layer)
    x = BatchNormalization()(x)
    x = Conv1D(64, 4, activation='relu', padding='same', strides=2)(x)
    x = BatchNormalization()(x)
    x = Conv1D(128, 4, activation='relu', padding='same', strides=2)(x)
    x = BatchNormalization()(x)
    x = Conv1D(256, 4, activation='relu', padding='same', strides=2)(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    output_layer = Dense(256, activation='softmax')(x)  # 256 units for each byte
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to train and evaluate a model for a specific byte of the keystream
def train_and_evaluate_byte_0(X_train, y_train, X_test, y_test):
    byte_index = 0
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_1(X_train, y_train, X_test, y_test):
    byte_index = 1
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

# Repeat similar function definitions for bytes 2 through 9

def train_and_evaluate_byte_2(X_train, y_train, X_test, y_test):
    byte_index = 2
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_3(X_train, y_train, X_test, y_test):
    byte_index = 3
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_4(X_train, y_train, X_test, y_test):
    byte_index = 4
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_5(X_train, y_train, X_test, y_test):
    byte_index = 5
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_6(X_train, y_train, X_test, y_test):
    byte_index = 6
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_7(X_train, y_train, X_test, y_test):
    byte_index = 7
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_8(X_train, y_train, X_test, y_test):
    byte_index = 8
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy

def train_and_evaluate_byte_9(X_train, y_train, X_test, y_test):
    byte_index = 9
    model = build_model()
    y_train_byte = to_categorical(y_train[:, byte_index], num_classes=256)
    y_test_byte = to_categorical(y_test[:, byte_index], num_classes=256)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    X_train_gpu = tf.convert_to_tensor(X_train_reshaped)
    y_train_gpu = tf.convert_to_tensor(y_train_byte)
    X_test_gpu = tf.convert_to_tensor(X_test_reshaped)
    y_test_gpu = tf.convert_to_tensor(y_test_byte)
    model.fit(X_train_gpu, y_train_gpu, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping], verbose=1)
    loss, accuracy = model.evaluate(X_test_gpu, y_test_gpu, verbose=0)
    return accuracy



Num GPUs Available:  1
2.10.1


In [ ]:
print('\nTraining for byte 9...')
accuracy = train_and_evaluate_byte_9(X_train, y_train, X_test, y_test)
#accuracies.append(accuracy)
print(f'Accuracy for byte 9: {accuracy * 100:.2f}%')


Training for byte 9...
